#### 0. Change directory

In [1]:
# Reference: https://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script/15622021#15622021
import sys
sys.path.append(r'S:\\Grid_Orientation\\LISFLOOD_FP\\Modelling')

#### 1. Packages

In [2]:
from folder import *
from randomisation import random_values_generation
from dataPreparation import download_lidar, \
                            padding_combination, \
                            dem_raster_reference, \
                            terrain_shading             # For lidar downloading, padding calculation, dem reference generation, and terrain shading

from transformation import center_calculation, get_url, get_tile_files, \
                           MyPool, transformation_performance, transformation_parallelism # For transformation using multiprocessing

from rasterSimulation import raster_generation

from floodModel import tide_flow_data, bdy_generation, tideboundary_points, flood_simulation

# For creating plot
import matplotlib.pyplot

# For untransformation
from untransformation import untransformation_parallelism

import multiprocessing
from functools import partial


# For terrain shading
import os
import rioxarray as rxr
import xarray as xr
from osgeo import gdal

#### 2. Necessary variables

##### 2.1. Basic variables

In [3]:
# Construct a list of boundary
resolution = 10
number_pixel_x = 16 * 42
number_pixel_y = 16 * 28

xmin = 1767790
ymin = 5471400

In [4]:
xmax = xmin + resolution * number_pixel_x
ymax = ymin + resolution * number_pixel_y

# Boundary for the area of interest
boundary_1 = [xmin, ymin, xmax, ymax]

# Padding boundary for DEMs (use this boundary to create padding)
addition_2 = 16 * 3
boundary_2 = padding_combination(boundary_1, addition_2)

# Boundary for tiles (use this boundary to download LiDAR)
addition_3 = 16 * 3
boundary_3 = padding_combination(boundary_2, addition_3)

In [5]:
# LiDAR dataset name
lidar_name = "Wellington_2013"

# Chunks and processors information
size_of_chunk = 100
size_of_processor = 4

# Flow data
flow_path = fr"S:\\new_versions\\data\\flow.csv"

# Tide data
tide_path = fr"S:\\new_versions\\data\\tide.csv"

# date start and end
date_start = r"2005-01-05 00:00:00"
date_end = r"2005-01-07 00:00:00"

# Tide flow data
tide_flow_data(tide_path, flow_path, date_start, date_end)

# Extracting flowdepth rate (for flowdepth_extraction() function)
flowdepth_rate = 0

# Time to extract flowdepth (for flowdepth_extraction() function)
extract_name = "out.max"

##### 2.2. Preparing flood model parameters

In [6]:
# Create tide_flow data
tide_flow_data(tide_path, flow_path, date_start, date_end)

# River points
river_points = (1774477.413, 5471521.558)

# Create BDY file
domain_name = "Waikanae"
bdy_generation(resolution, domain_name)

# Create tide boundary
tideboundary_points(boundary_1, False)

##### 2.3. Preparing some flood model inputs

In [7]:
%%time
# # Download LiDAR
# download_lidar(boundary_3, lidar_name)

# # Reference DEM without padding
# dem_raster_reference(resolution, size_of_chunk, size_of_processor,
#                      boundary_1, lidar_name, "no_padding")

# # Reference DEM with padding
# dem_raster_reference(resolution, size_of_chunk, size_of_processor,
#                      boundary_2, lidar_name, "padding")

# # DEM for hillshade/terrain shade
# dem_raster_reference(2, 400, 6,
#                      boundary_1, lidar_name, "shading")

# # Terrain shading
# terrain_shading(50, 355)

# Calculate coordinates of center point
center_point = center_calculation(True)
center_x = center_point[0]                     # Extract x coordinate of center point
center_y = center_point[1]                     # Extract y coordinate of center point

# Transformation values
ran_trans = random_values_generation(
    1,
    5,
    [0, 90],
    [0, resolution/2],
    [0, resolution/2],
    'uniform',
    True
)

# Get url list file
url_list_file = get_url(lidar_name)
tile_files = get_tile_files(lidar_name)

CPU times: total: 797 ms
Wall time: 855 ms


#### 3. Execution

In [8]:
%%time
transformation_parallelism(
    center_x,
    center_y,
    lidar_name,
    tile_files,
    30,
    url_list_file,
    ran_trans,
    1
)

CPU times: total: 0 ns
Wall time: 20min 28s


In [9]:
%%time
# DEM CREATION
# List parameters
resolution_func = resolution
chunk_size_func = size_of_chunk
processor_func = size_of_processor
padding_func = boundary_2
lidar_dataset_name = lidar_name

# Design func
func = partial(
    raster_generation,
    resolution_func,
    chunk_size_func,
    processor_func,
    padding_func,
    lidar_dataset_name,
    center_x, center_y
)

# Design pool
pool = MyPool(5)
pool.map(func, [ran for ran in ran_trans])
pool.close()
pool.join()

CPU times: total: 15.6 ms
Wall time: 50min 15s


In [10]:
%%time
for ran_trans_i in ran_trans:
    flood_simulation(
        domain_name,
        river_points,
        center_x, center_y,
        fr"S:\\new_versions\\data",
        ran_trans_i
    )

CPU times: total: 219 ms
Wall time: 48min 43s


In [11]:
%%time
untransformation_parallelism(
    extract_name,
    ran_trans,
    30
)

CPU times: total: 31.2 ms
Wall time: 56.2 s
